In [ ]:
import requests
import json
import pickle



def get_bne_wikidata(max_number=100):
    key = "bne"
    value = "wikidata"
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
    SELECT ?bne ?wikidata
    WHERE { 
        ?wikidata wdt:P950 ?bne .
    }
    LIMIT """+str(max_number)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    json_data = response.json()['results']['bindings']
    return [(item[key]['value'], item[value]['value']) for item in json_data]


datos = get_bne_wikidata()
#for key,value in datos:
    #print(key,"\t",value)

f = open("wikidatamap.bin", "wb+")
pickle.dump(datos, f)

In [ ]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
query2 = """PREFIX wikibase: <http://wikiba.se/ontology>
construct { ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }
WHERE { ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
    ?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
} LIMIT 10000 OFFSET 1880000
"""
datos.load_dataset_from_query(query2, only_uri=False)


#datos.load_dataset_from_nlevels(5, extra_params="LIMIT 100")
datos.show(verbose=False)
dataset1 = datos.train_split()

#     print(sub,dataset1[sub])
datos.save_to_binary("wikidata_2.bin")

# datos.subs

#print(len(jsonlist))
#jsonlist

In [ ]:
import numpy as np
rel = [i[2] for i in datos.subs]
print(len(datos.relations_dict))
print(len(datos.relations))
print(len(datos.entities_dict))
print(len(datos.entities))
rel = np.array(rel)
print(np.sort(rel))
data = np.matrix(datos.subs)
print(data[:,2])

print('Splitting the data into training and validation sets ...')
print(data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
nb_validation_samples = int(0.2 * data.shape[0])


x_train = data[:-nb_validation_samples]
x_val = data[-nb_validation_samples:-int(nb_validation_samples/2)]
x_test = data[-int(nb_validation_samples/2):]

print(x_val.shape,"\n\n\n",x_test.shape,"\n\n\n",x_train.shape)
print(x_test.shape[0]+x_train.shape[0]+x_val.shape[0])


In [13]:
import importlib
import pickle
import kgeserver.dataset as dataset
importlib.reload(dataset)


datos = dataset.Dataset()
fb_test = open("FB15k/freebase_mtr100_mte100-test.txt", "r")
fb_train = open("FB15k/freebase_mtr100_mte100-train.txt", "r")
fb_valid = open("FB15k/freebase_mtr100_mte100-valid.txt", "r")
datos.load_dataset_from_csv(fb_train, "\t")
datos.load_dataset_from_csv(fb_test, "\t")
datos.load_dataset_from_csv(fb_valid, "\t")

datos.show(verbose=False)



14951 entities, 1345 relations, 592213 tripletas


In [ ]:
import requests

def get_json_offset(n, count):
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
construct {{ ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }}
WHERE {{ ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
}} LIMIT {0} OFFSET {1}
""".format(count, n*count)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    if response.status_code is not 200:
        return False, "Error occurred on http request. Code"+str(response.status_code)
    json_data = response.json()['results']['bindings']
    return json_data

all_json = []
for query in range(0,30):
    json1 = get_json_offset(query,20000)
    all_json = all_json + json1
    


In [1]:
import importlib
import kgeserver.dataset as dataset
import kgeserver.wikidata_dataset as wikidata_dataset
import pickle
importlib.reload(dataset)
importlib.reload(wikidata_dataset)
from datetime import datetime

dtset = wikidata_dataset.WikidataDataset()


#dataset.load_entire_dataset(1)
dtset.load_dataset_recurrently(3, verbose=2, limit_ent=2000)
dtset.show()



Scanning level 1 with 2000 elements
Enter S to show status: s
Elapsed time: 1s. Depth 1 of 3. Entities scanned: 4.35% (87 of 2000) Active threads: 36
Waiting all threads to end
Scanning level 2 with 2000 elements
Waiting all threads to end
Scanning level 3 with 2000 elements
Waiting all threads to end
38614 entities, 382 relations, 93288 tripletas


In [22]:
print(dtset.relations[8])
dtset.save_to_binary("wikidata_25k.bin")

P31


True

In [ ]:
import importlib
import dataset
import algorithm
import skge
importlib.reload(dataset)
importlib.reload(algorithm)


dt = dataset.Dataset()

dt.load_from_binary("wn18.bin")

al = algorithm.Algorithm(dt)

models = models = al.find_best(ncomps=[50], margins=[2], model_types=[skge.TransE])

In [ ]:
import importlib
import dataset
import algorithm
import skge
importlib.reload(dataset)
importlib.reload(algorithm)


dt = dataset.Dataset()

dt.load_from_binary("wn18.bin")

al = algorithm.Algorithm(dt)

models = models = al.find_best(ncomps=[150], margins=[0.2], model_types=[skge.HolE])